In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
len (test)

In [ ]:
test[test["id"] == "423A1CA112E2"]

In [ ]:
id_423A1CA112E2 = test[test["id"] == "423A1CA112E2"]

In [ ]:
id_423A1CA112E2

In [ ]:
with open('../input/feedback-prize-2021/train/423A1CA112E2.txt' ,"r") as f:
    contents = f.read()
    #print(contents)
    
print ("Lead:\n" , contents[8:229] )
print ("Position 1:\n" , contents[8:229] )
print ("Lead:\n" , contents[8:229] )

In [ ]:
with open('../input/feedback-prize-2021/train/423A1CA112E2.txt' ,"r") as f:
    contents = f.read()
count=1
for i in range(len(id_423A1CA112E2)):
    discourse_id = id_423A1CA112E2.loc[i]["discourse_id"]
#     start = id_423A1CA112E2.loc[i]["discourse_start"]
#     end = id_423A1CA112E2.loc[i]["discourse_end"]
    typ = id_423A1CA112E2.loc[i]["discourse_type"]
    prdstring = id_423A1CA112E2.loc[i]["predictionstring"]
#     arg = contents[int(start):int(end)]
    discourse_text = id_423A1CA112E2.loc[i]["discourse_text"]
    length = len(discourse_text.rstrip().split())
#     "".join(discourse_text.rstrip().lstrip())
    
    
    print (str (discourse_id) +"\n" + typ + "\n"+ discourse_text  + "\n" + prdstring+"\n" , "range:"+str (count)+ "-" + str(count+length-1) + "\n")
    print (discourse_text.split(" "))
    
    count+=length

# ****Read the Data****

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
test.shape

# Preprocessing 

In [ ]:
import string
import re
from nltk.corpus import stopwords
from nltk import download
from nltk.tokenize import RegexpTokenizer

download ("stopwords")
stp_words = stopwords.words("english")
stop_words = set ([w.replace('\'', '') for w in stp_words] + stp_words)
tokenizer = RegexpTokenizer(r'\w+')

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#defining function for tokenization
def tokenization(text):
    tokens = tokenizer.tokenize(text)
    return tokens

def preprocess(text):
    text = remove_punctuation(text)
    text = tokenization(text)
    text = [word.lower() for word in text]
    
    preprocessed_sentence = []
    for word in text:
        if word not in stop_words:
            preprocessed_sentence.append(word)
    
    return " ".join (preprocessed_sentence)

In [ ]:
#storing the preprocessed text
test['pre_discourse_text']= test['discourse_text'].apply(lambda x:preprocess(x))

# Document representation

In [ ]:
import gensim.downloader as gsm
modelW2v = gsm.load("word2vec-google-news-300")

In [ ]:
from cuml.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings = tfidf.fit_transform( train_text_df.text ).toarray()

In [ ]:
from gensim.similarities import WmdSimilarity
from pyemd import emd
from gensim.test.utils import common_texts
from time import time

# Run WMD and find the most 10 similar documents.

start = time()

instance = WmdSimilarity(test['pre_discourse_text'], modelW2v)
X =[]
for document in test['pre_discourse_text']:
    X.append (instance[document])# A query is simply a "look-up" in the similarity class.

print ('seconds to complete:', (time() - start)/60)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cudf ,cuml
import cupy as cp
#print('RAPIDS',cudf.__version__)

In [ ]:
ids, texts = [], []
listOfFiles = list(os.listdir('../input/feedback-prize-2021/train'))

for f in tqdm(listOfFiles):
    ids.append(f.replace('.txt', ''))
    texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
    
    
trainData = cudf.DataFrame({'id': ids, 'text': texts})


#df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test = cudf.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
#test['pre_discourse_text']= test['discourse_text'].apply(lambda x:preprocess(x))

test['pre_discourse_text']= [preprocess(x) for x in test['discourse_text'].to_pandas()]

In [ ]:
type(test)

In [ ]:
from cuml.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', binary=True, max_features=50_000)



transformed_documents1 = tfidf.fit_transform(test[:40_000].pre_discourse_text)
transformed_documents2 = tfidf.fit_transform(test[40_000:80_000].pre_discourse_text)
transformed_documents3 = tfidf.fit_transform(test[80_000:120_000].pre_discourse_text)
transformed_documents4 = tfidf.fit_transform(test[120_000:144_293].pre_discourse_text)

In [ ]:
#transformed_documents = tfidf.fit_transform(test.pre_discourse_text)

In [ ]:
transformed_documents2.shape

# T-sne

In [ ]:
#from cuml.manifold import TSNE
from cuml import TSNE
#import seaborn as sns
#from sklearn.preprocessing import StandardScaler

#frame , Rscore = calculateAC('average' , 'cosine' , 11,  DistanceW2v)

# scaler = StandardScaler()
# scaler.fit(DistanceW2v)
# X_scale = scaler.transform(DistanceW2v)

# tsne = TSNE(n_components=2, metric="precomputed",
#               learning_rate = 200, verbose = 1,perplexity = 200)

# results = tsne.fit_transform(transformed_documents)
# embedding = cp.asnumpy(results)

from cuml import UMAP
umap = UMAP()
embed_2d1 = umap.fit_transform(transformed_documents1)
embed_2d1 = cupy.asnumpy( embed_2d1 )

embed_2d2 = umap.fit_transform(transformed_documents2)
embed_2d2 = cupy.asnumpy( embed_2d2 )

embed_2d3 = umap.fit_transform(transformed_documents3)
embed_2d3 = cupy.asnumpy( embed_2d3 )

embed_2d4 = umap.fit_transform(transformed_documents4)
embed_2d4 = cupy.asnumpy( embed_2d4 )


#tsne_df_scale = pd.DataFrame(tsne_scale_results, columns=['tsne1', 'tsne2'])

# plt.figure(figsize = (10,10))
# #plt.scatter(tsne_df_scale.iloc[:,0],
# #            tsne_df_scale.iloc[:,1],
# #            alpha=0.25, facecolor='lightslategray')

# sns.scatterplot(x = tsne_df_scale.iloc[:,0],
#                 y = tsne_df_scale.iloc[:,1],
#                 hue=frame['Cluster'], palette='Set1', s=100, alpha=0.6).set_title('Cluster Via tSNE Scaled Data',
#                                                                                    fontsize=15)
# plt.xlabel('tsne1')
# plt.ylabel('tsne2')
# plt.show()

In [ ]:
transformed_documents.shape

In [ ]:
embed = umap.fit_transform(transformed_documents)
embed = cupy.asnumpy(embed)

In [ ]:
type(embed_2d)

In [ ]:
embed_2d = np.concatenate((embed_2d1, embed_2d2), axis=0) #embed_2d1.a # embed_2d2 + embed_2d3 + embed_2d4  
embed_2d = np.concatenate((embed_2d, embed_2d3), axis=0)
embed_2d = np.concatenate((embed_2d, embed_2d4), axis=0)

In [ ]:
embed_2d.shape

In [ ]:
set(test['discourse_type'].to_pandas())

In [ ]:
classes={"Claim":1 ,
         "Concluding Statement":2 ,
         "Counterclaim":3,
         "Evidence":4,
         "Lead":5,
         "Position":6, "Rebuttal":7 
        }
test['classes']= [classes[x] for x in test['discourse_type'].to_pandas()]

In [ ]:
test.head(1)

In [ ]:
test.classes.to_pandas().shape

In [ ]:
embed_2d.shape

In [ ]:
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split

# iris = datasets.load_iris()
X, y = embed_2d, test.classes.to_pandas() # [:160000] #, np.zeros((160000,)) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                    random_state = 0)

knn = neighbors.KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

# What kind of iris has 3cm x 5cm sepal and 4cm x 2cm petal?
#print(iris.target_names[knn.predict([[3, 5, 4, 2]])])

In [ ]:
type(X)

In [ ]:
y_test

In [ ]:
print()

# acc =[1 if x else 0 for x in (knn.predict(X_test) == y_test)]
# sum(acc)/len(acc)    

knn.score(X_test , y_test)

In [ ]:
test.loc[1]

In [ ]:
ids, texts = [], []
listOfFiles = list(os.listdir('../input/feedback-prize-2021/test'))

for f in tqdm(listOfFiles):
    ids.append(f.replace('.txt', ''))
    ll = open('../input/feedback-prize-2021/test/' + f, 'r').read().split("\n")
    texts.append(ll)
    #ll = texts
    
testData = cudf.DataFrame({'id': ids, 'text': texts})


# #df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
# test = cudf.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
listOfFiles

# Trying spell-checker and word2vec

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cudf ,cuml
import cupy as cp
import os
#print('RAPIDS',cudf.__version__)

In [ ]:
ids, texts = [], []
listOfFiles = list(os.listdir('../input/feedback-prize-2021/train'))

for f in tqdm(listOfFiles):
    ids.append(f.replace('.txt', ''))
    texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
    
    
trainData = cudf.DataFrame({'id': ids, 'text': texts})


#df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
train = cudf.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
train.head()

In [ ]:
import string
import re
from nltk.corpus import stopwords
from nltk import download
from nltk.tokenize import RegexpTokenizer

download ("stopwords")
stp_words = stopwords.words("english")
stop_words = set ([w.replace('\'', '') for w in stp_words] + stp_words)
tokenizer = RegexpTokenizer(r'\w+')

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#defining function for tokenization
def tokenization(text):
    tokens = tokenizer.tokenize(text)
    return tokens

def preprocess(text):
    text = remove_punctuation(text)
    text = tokenization(text)
    text = [word.lower() for word in text]
    
    preprocessed_sentence = []
    for word in text:
        if word not in stop_words:
            preprocessed_sentence.append(word)
    
    return " ".join (preprocessed_sentence)

In [ ]:
# !pip install pyspellchecker

In [ ]:
train['pre_discourse_text']= [preprocess(x) for x in train['discourse_text'].to_pandas()]

In [ ]:
classes={"NaN": ,
         "Claim":1 ,
         "Concluding Statement":2 ,
         "Counterclaim":3,
         "Evidence":4,
         "Lead":5,
         "Position":6,
         "Rebuttal":7 
        }
test['classes']= [classes[x] for x in test['discourse_type'].to_pandas()]

In [ ]:
train.columns

In [ ]:
ll = train[
    train["discourse_type"] != "Claim"
]

ll = ll [
        ll["discourse_type"] != "Concluding Statement" 
    ]

ll = ll [
        ll["discourse_type"] != "Evidence" 
    ]


ll = ll [
        ll["discourse_type"] != "Lead" 
    ]

ll = ll [
        ll["discourse_type"] != "Position" 
    ]

ll = ll [
        ll["discourse_type"] != "Rebuttal" 
    ]

ll = ll [
        ll["discourse_type"] != "Counterclaim" 
    ]

# train["discourse_type"] != "Counterclaim" and 
# train["discourse_type"] != "Evidence" and
# train["discourse_type"] != "Lead" and
# train["discourse_type"] != "Position" and
#     train["discourse_type"] != "Rebuttal"

In [ ]:
ll 

In [ ]:
import gensim.downloader as gsm
modelW2v = gsm.load("word2vec-google-news-300")

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
transformed_documents = sbert_model.encode (test['pre_discourse_text'].to_pandas())

In [ ]:
print (transformed_documents.shape)

In [ ]:
from cuml import UMAP
umap = UMAP()

embed_2d = umap.fit_transform(transformed_documents)
embed_2d = cp.asnumpy(embed_2d)

In [ ]:
#from cuml.manifold import TSNE
from cuml import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

#from sklearn.preprocessing import StandardScaler

#frame , Rscore = calculateAC('average' , 'cosine' , 11,  DistanceW2v)

# scaler = StandardScaler()
# scaler.fit(DistanceW2v)
# X_scale = scaler.transform(DistanceW2v)

# tsne = TSNE(n_components=2, metric="precomputed",
#               learning_rate = 200, verbose = 1,perplexity = 200)

# results = tsne.fit_transform(transformed_documents)
# embedding = cp.asnumpy(results)



tsne_df_scale = pd.DataFrame(embed_2d, columns=['tsne1', 'tsne2'])

plt.figure(figsize = (10,10))
plt.scatter(tsne_df_scale.iloc[:,0],
           tsne_df_scale.iloc[:,1],
           alpha=0.25, facecolor='lightslategray')

sns.scatterplot(x = tsne_df_scale.iloc[:,0],
                y = tsne_df_scale.iloc[:,1],
                hue=test['classes'].to_pandas(),
                palette='Set1', s=100, alpha=0.6).set_title('Cluster Via tSNE Scaled Data',fontsize=15)

plt.axis([0, 10, 0, 10])
plt.xlabel('tsne1')
plt.ylabel('tsne2')
plt.show()

In [ ]:
embed_2d.shape

In [ ]:
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split

X, y = embed_2d, test.classes.to_pandas() # [:160000] #, np.zeros((160000,)) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                    random_state = 0)

In [ ]:
maxScoreCluster=-1
clusterNum = -1
for i in range (2,100):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    
    score = knn.score(X_test , y_test)
    if score > maxScoreCluster:
        maxScoreCluster = score 
        clusterNum = i

print (clusterNum," : ", maxScoreCluster)

In [ ]:
# id1, text1 = [], []
# listOfFiles = list(os.listdir('../input/feedback-prize-2021/test'))

# for f in tqdm(listOfFiles):
#     t = open('../input/feedback-prize-2021/test/' + f, 'r').read()
    
#     for sent in t.split("\n"):
#         if sent != '':
#             id1.append(f.replace('.txt', ''))
#             text1.append(sent)
        
    
# testData = cudf.DataFrame({'id': id1, 'text': text1})


# df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
# #test = cudf.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
# testData
# testData['pre_discourse_text']= [preprocess(x) for x in testData['text'].to_pandas()]